## 1 PyTorch中的Transformer

在PyTorch中，Transformer算法是属于“构建循环神经网络的元素”，而非“成熟神经网络”，因此Transformer是位于PyTorch.nn这个基本模块下。但为什么PyTorch中的Transformer结构是位于nn，而不是属于成熟神经网络呢？**事实上，在PyTorch中并没有完整的Transformer架构，只有用于构建Transformer的各个层**。我们一起来看一下。

<center><img src="https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/transformer/image-1.png" alt="描述文字" width="400">

在torch.nn模块下，存在**服务于Transformer架构的各类神经网络层和模型**，我们来看一下——

- **nn.Transformer**

`nn.Transformer`封装了Transformer中的包含编码器（Encoder）和解码器（Decoder）。如下图所示，它对Encoder和Decoder两部分的包装，它并没有实现输入中的Embedding和Positional Encoding和最后输出的Linear+Softmax部分。

<center><img src="https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/transformer/image-37.png" alt="描述文字" width="400">

- **分割的编码器与解码器**

`nn.TransformerEncoderLayer`与`nn.TransformerDecoderLayer`: 这两个类表示Transformer编码器的单层和解码器的单层。它包含了自注意力机制（self-attention）和前馈网络（feedforward network），以及必要的归一化和残差连接。

`nn.TransformerEncoder`与`nn.TransformerDecoder`: 这两个类是Transformer编码器的实现和解码器的实现，其中`nn.TransformerEncoder`包含了多个nn.TransformerEncoderLayer层的堆叠，`nn.TransformerDecoder`包含了多个nn.TransformerDecoderLayer层的堆叠。

<center><img src="https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/transformer/image-38.png" alt="描述文字" width="800">

除此之外，我们还有：

`nn.MultiheadAttention`: 这个模块实现了多头注意力机制，这是Transformer模型的核心组件之一。多头注意力允许模型在不同的位置同时处理来自序列不同部分的信息，这有助于捕捉序列内的复杂依赖关系。

`nn.LayerNorm`: 层归一化（Layer Normalization）通常用在Transformer的各个子层的输出上，有助于稳定训练过程，并且提高了训练的速度和效果。

`nn.PositionalEncoding`: 虽然PyTorch的nn模块中没有这个类，但在使用Transformer时通常会添加一个位置编码层来给模型提供关于词汇在序列中位置的信息，因为Transformer本身不具备处理序列顺序的能力。

`nn.Embedding`：一个预训练好的语义空间，它将每个标记（如单词、字符等）映射到一个高维空间的向量。这使得模型能够处理文本数据，并为每个唯一的标记捕获丰富的语义属性。嵌入层通常是自然语言处理模型的第一层，用于将离散的文本数据转化为连续的向量表示。其输入是索引列表，输出是对应的嵌入向量。

`nn.Transformer.generate_square_subsequent_mask`：掩码函数。用于生成一个方形矩阵，用作Transformer模型中自注意力机制的上三角遮罩。这个遮罩确保在序列生成任务中，例如语言模型中，任何给定的元素只会考虑到序列中先于它的元素（即它只能看到过去的信息，不能看到未来的信息）。这种掩码通常在解码器部分使用，防止在预测下一个输出时“作弊”。具体来说，该函数创建了一个方阵，其中对角线及其以下的元素为0（表示可以“看到”这些位置的元素），其余元素为负无穷大（在softmax之前应用，表示位置被屏蔽，不应该有注意力权重）。

In [72]:
import torch.nn as nn

In [75]:
nn.Transformer.generate_square_subsequent_mask(5) # 5指的是target的维度

tensor([[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]])

不难发现，torch.nn下面配置了一系列构成transformer的元素，但是却没有构成完整的Transformer算法。这是为什么呢？事实上，在NLP的世界中，不同的任务会对Transformer架构提出不同的要求，编码器和解码器是设计成可以独立或一起使用的组件。它们可以根据不同的NLP任务需求进行组合，以适应各种场景。

- **只使用编码器的任务**

编码器部分的任务是从输入数据中提取特征。编码器通常用于不需要生成新文本序列的任务，比如：

> 文本分类：如情感分析，垃圾邮件检测等，输入一个文本序列，编码器提取特征后进行分类。

> 命名实体识别（Named Entity Recognition, NER）：在给定文本中识别出实体（如人名、地点等），这也是分类问题的一种，可以用编码器提取文本的特征。

> 句子相似度：判断两个句子是否相关或相似度如何，可以通过编码器提取句子特征后计算相似度。

- **只使用解码器的任务**

解码器部分专注于生成新文本。通常使用自回归方式，基于之前的输出生成下一个词。适用于：

> 文本生成：如GPT系列，只使用解码器生成文本，例如故事续写、文章生成等。

> 文本续写或预测：训练解码器来预测下一个单词或字符。

- **编码器和解码器都使用的任务**：

当任务涉及到理解输入序列并生成新的输出序列时，编码器和解码器会联合使用，比如：

> 机器翻译：编码器负责理解源语言，解码器负责生成目标语言。

> 文本摘要：编码器理解原始文本，解码器生成摘要。

> 问答系统：编码器理解给定的问题和背景材料，解码器生成答案。

这种分离使用编码器或解码器的能力使得Transformer架构极为灵活，可以通过调整来适应各种不同的NLP任务需求。而具体的架构选择（编码器、解码器或两者兼用）取决于我们试图解决的问题类型。

- **nn.Transformer的参数构造详解**

在nn.Transformer中，我们有如下的参数可以使用：

| 参数名| 解释 |
| ---- | ---- |
| d_model| Encoder和Decoder输入参数的特征维度。也就是词向量的维度。默认为512 |
| nhead| 多头注意力机制中，head的数量。默认值为8。 |
| num_encoder_layers | TransformerEncoderLayer的数量。该值越大，网络越深，网络参数量越多，计算量越大。默认值为6 |
| num_decoder_layers | TransformerDecoderLayer的数量。该值越大，网络越深，网络参数量越多，计算量越大。默认值为6 |
| dim_feedforward | Feed Forward层（Attention后面的全连接网络）的隐藏层的神经元数量。该值越大，网络参数量越多，计算量越大。默认值为2048 |
| dropout | 内dropout值。默认值为0.1 |
| activation | 内Feed Forward层的激活函数。取值可以是string(“relu” or “gelu”)或者一个一元可调用的函数。默认值是relu |
| custom_encoder | 自定义Encoder。若你不想用官方实现的TransformerEncoder，你可以自己实现一个。默认值为None |
| custom_decoder | 自定义Decoder。若你不想用官方实现的TransformerDecoder，你可以自己实现一个。 |
| layer_norm_eps| Add&Norm层中，BatchNorm的eps参数值。默认为1e-5|
| batch_first | batch维度是否是第一个。如果为True，则输入的shape应为(batch_size, 词数，词向量维度)，否则应为(词数, batch_size, 词向量维度)。默认为False。这个要特别注意，因为大部分人的习惯都是将batch_size放在最前面，而这个参数的默认值又是False，所以会报错。|
| norm_first | 是否要先执行norm。例如，在图中的执行顺序为 Attention -> Add -> Norm。若该值为True，则执行顺序变为：Norm -> Attention -> Add。|

<center><img src="https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/transformer/image-1.png" alt="描述文字" width="400">

In [76]:
import torch
from torch.nn import Transformer
import torch.nn as nn

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformer 参数
d_model = 512  # Embedding size
nhead = 8  # 头的数量
num_encoder_layers = 6  # 编码器层的数量
num_decoder_layers = 6  # 解码器层的数量
dim_feedforward = 2048  # 前馈网络模型的尺寸
dropout = 0.1  # Dropout 比例

# 实例化 nn.Transformer 模型
transformer_model = Transformer(d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, dropout).to(device)

# 示例输入数据 (seq_len, batch_size, d_model)
src = torch.rand(10, 32, d_model).to(device)  # 编码器输入
tgt = torch.rand(10, 32, d_model).to(device)  # 解码器输入
src_mask = transformer_model.generate_square_subsequent_mask(src.size(0)).to(device)
tgt_mask = transformer_model.generate_square_subsequent_mask(tgt.size(0)).to(device)

# 正向传播
output = transformer_model(src, tgt, src_key_padding_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None)
print(output)  # 输出结果 (seq_len, batch_size, d_model)

tensor([[[ 6.7511e-02,  1.1720e+00,  1.2293e-01,  ...,  7.8842e-01,
           1.7112e-01,  9.6802e-01],
         [-7.9600e-01,  9.0369e-02, -2.8433e-01,  ...,  8.8442e-01,
           4.5074e-01,  1.2281e+00],
         [-5.1751e-01, -1.5499e-01, -6.7926e-01,  ...,  1.7317e-01,
           7.3845e-01,  1.1859e+00],
         ...,
         [-5.6413e-01,  8.6297e-02,  2.1021e-01,  ...,  2.6536e-01,
           5.3328e-01,  1.0273e+00],
         [ 4.5727e-01, -1.8545e-01,  8.5030e-01,  ...,  4.5394e-01,
           1.2406e+00,  7.0520e-04],
         [-5.8062e-01, -3.7102e-01,  3.1228e-01,  ..., -1.8074e-02,
          -9.9500e-01,  3.6074e-01]],

        [[-3.6611e-01, -1.4964e-01, -5.7366e-01,  ...,  1.1742e+00,
           4.5474e-01,  1.5793e+00],
         [-7.1758e-01, -1.1414e-01, -1.4992e-01,  ...,  9.3404e-01,
           6.8837e-01,  1.4203e-01],
         [-1.1819e+00, -1.3623e-01, -8.2696e-01,  ...,  1.1469e+00,
           1.5560e+00,  1.9745e+00],
         ...,
         [ 3.4972e-01, -3

## 2 Transformer在传统NLP领域的应用

Transformer的模块化构件，使得它可以拆分使用或者“魔改”自由拼装。通常不同的NLP任务上会使用Transformer架构的不同部分，很多更优的模型也是在Transformer的基础上调整改造。
我们只需要理解：**Encoder通常用于提取特征，Decoder通常用于生成内容**。

* 同时使用encoder和decoder：机器翻译、文本摘要
* 只使用encoder：文本分类（bert）、问答系统、实体命名识别、填充任务
* 只使用decoder：文本生成（GPT）

以下代码提供了各个任务的基础架构，但为了完全实现这些模型，还需要包括数据预处理、mask的创建、训练循环、损失函数计算、优化器配置等。这些代码片段主要是为了展示不同任务中 nn.Transformer 结构的不同使用方式。实际使用时，每个任务还需要细化和调整以适应具体的需求和数据集。在正式的《深度学习实战》第七期课程当中，我们将详细讲解这些内容。

### 2.1 机器翻译任务
同时使用Transformer的encoder和decoder。

In [ ]:
import torch
from torch import nn
from torch.nn import Transformer

# 假设一些超参数和数据预处理已经完成
src_vocab_size = 1000  # 源语言词汇量
tgt_vocab_size = 1000  # 目标语言词汇量
embedding_size = 512  # 嵌入维度
nhead = 8  # 多头注意力中的头数
num_encoder_layers = 6  # 编码器层数
num_decoder_layers = 6  # 解码器层数
dropout = 0.1  # dropout比率

class TranslationModel(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, embedding_size, nhead, num_encoder_layers, num_decoder_layers, dropout):
        super(TranslationModel, self).__init__()
        self.transformer = Transformer(d_model=embedding_size, nhead=nhead, num_encoder_layers=num_encoder_layers, num_decoder_layers=num_decoder_layers, dropout=dropout)
        self.src_tok_emb = nn.Embedding(src_vocab_size, embedding_size)
        self.tgt_tok_emb = nn.Embedding(tgt_vocab_size, embedding_size)
        self.positional_encoding = nn.Parameter(torch.zeros(1, 5000, embedding_size))  # 假定最大句子长度为 5000
        self.generator = nn.Linear(embedding_size, tgt_vocab_size)
        
    #----必填参数----
    #src：Encoder的输入。也就是将token进行Embedding和Positional Encoding之后的tensor。必填参数。Shape为(batch size，sequence length，embedding size)
    #tgt:与src同理，Decoder的输入。 必填参数。Shape为(sequence length，embedding size)
    
    #----可选参数----
    #src_mask：对src进行mask。不常用。Shape为(sequence length, sequence length)
    #tgt_mask：对tgt进行mask。常用。Shape为(sequence length, sequence length) 
    #src_key_padding_mask：对src的token进行mask. 常用。Shape为(batch size, sequence length) 
    #tgt_key_padding_mask：对tgt的token进行mask。常用。Shape为(batch size, sequence length) 
    
    def forward(self, src, tgt, src_mask = None, tgt_mask = None, memory_mask = None, src_key_padding_mask = None, tgt_key_padding_mask = None,memory_key_padding_mask = None):
        src_emb = self.src_tok_emb(src) + self.positional_encoding[:, :src.size(1)]
        tgt_emb = self.tgt_tok_emb(tgt) + self.positional_encoding[:, :tgt.size(1)]
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None, src_key_padding_mask, tgt_key_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

# 假定 src 和 tgt 已经是预处理并被转换成 tensor 的索引
# src = [batch_size, src_len]
# tgt = [batch_size, tgt_len]

model = TranslationModel(src_vocab_size, tgt_vocab_size, embedding_size, nhead, num_encoder_layers, num_decoder_layers, dropout)

# 接下来你需要定义mask和padding mask，并且运行模型
# 比如 src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, memory_key_padding_mask
# 输出会是一个[batch_size, tgt_len, tgt_vocab_size]的tensor

### 2.2 文本分类
文本分类通常只使用encoder部分，我们以此为例看一下encoder部分如何单独使用。

In [ ]:
import torch
from torch import nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer

# 假设超参数和数据预处理已经完成
vocab_size = 1000  # 词汇量
embedding_size = 512  # 嵌入维度
nhead = 8  # 多头注意力的头数
num_encoder_layers = 6  # 编码器层数
dropout = 0.1  # dropout比率
num_classes = 10  # 类别数

class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embedding_size, nhead, num_encoder_layers, dropout, num_classes):
        super(TextClassificationModel, self).__init__()
        #只使用了Transformer的Encoder部分
        encoder_layers = TransformerEncoderLayer(d_model=embedding_size, nhead=nhead, dropout=dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layer=encoder_layers, num_layers=num_encoder_layers)

        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.positional_encoding = nn.Parameter(torch.zeros(1, 5000, embedding_size))  # 假定最大句子长度为 5000
        self.linear = nn.Linear(embedding_size, num_classes)
        
    #----必填参数----
    #src：Encoder的输入。也就是将token进行Embedding和Positional Encoding之后的tensor。必填参数。Shape为(batch size，sequence length，embedding size)
    
    #----可选参数----
    #src_mask：对src进行mask。不常用。Shape为(sequence length, sequence length)
    #src_key_padding_mask：对src的token进行mask. 常用。Shape为(batch size, sequence length) 

    def forward(self, src, src_mask, src_padding_mask):
        src_emb = self.embedding(src) + self.positional_encoding[:, :src.size(1)]
        encoded_src = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        # 假设我们只关心输入序列的第一个元素
        return self.linear(encoded_src[:, 0])

# 假定 src 已经是预处理并被转换成 tensor 的索引
# src = [batch_size, src_len]

model = TextClassificationModel(vocab_size, embedding_size, nhead, num_encoder_layers, dropout, num_classes)

# 接下来你需要定义mask和padding mask，并且运行模型
# 比如 src_mask, src_padding_mask
# 输出会是一个[batch_size, num_classes]的tensor

对于文本分类的任务只使用encoder部分，通常会在encoder后直接连接全连接层，将编码器的输出转换为最终的分类标签。

BERT（Bidirectional Encoder Representations from Transformers，Transformers双向编码表示）就是只使用Transformer的encoder部分的重要例子。

### 2.3 类似GPT的文本生成式模型
目前AI前沿的GPT大家都知道它是基于Transformer的，可是它是具体如何实现的呢？我们来看一下简单的文本生成式模型是什么样的结构。

In [ ]:
import torch
from torch import nn

class GPTLikeModel(nn.Module):
    def __init__(self, vocab_size, embedding_size, nhead, num_decoder_layers, dropout):
        super(GPTLikeModel, self).__init__()
        self.embedding_size = embedding_size
        self.embeddings = nn.Embedding(vocab_size, embedding_size)
        self.positional_encodings = nn.Parameter(torch.zeros(1, 1024, embedding_size))
        
        #只使用了Transformer的Decoder部分
        self.transformer_decoder_layer = nn.TransformerDecoderLayer(d_model=embedding_size,  nhead=nhead, dropout=dropout)
        self.transformer_decoder = nn.TransformerDecoder(self.transformer_decoder_layer, num_layers=num_decoder_layers)

        self.output_layer = nn.Linear(embedding_size, vocab_size)
    
    #生成上三角矩阵掩码
    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask
    
    #----必填参数---- 
    #tgt:与src同理，Decoder的输入。 必填参数。也就是将token进行Embedding和Positional Encoding之后的tensor。Shape为(batch size，sequence length，embedding size)
    #----可选参数----
    #tgt_mask：对tgt进行mask。常用。Shape为(sequence length, sequence length) 
    #tgt_key_padding_mask：对tgt的token进行mask。常用。Shape为(batch size, sequence length) 
    #memory_key_padding_mask：对tgt的token进行mask。不常用。Shape为(batch size, sequence length) 
    def forward(self, tgt, memory_key_padding_mask):
        input_mask = self.generate_square_subsequent_mask(input.size(0)).to(input.device)
        embeddings = self.embeddings(input) * math.sqrt(self.embedding_size)
        embeddings += self.positional_encodings[:, :tgt.size(0), :]
        transformer_output = self.transformer_decoder(embeddings, memory, tgt_mask=tgt_mask)
        output = self.output_layer(transformer_output)
        return output

    def generate(self, input, max_length):
        memory = None
        generated = input
        for _ in range(max_length):
            
            #进行一个字的生成
            output = self.forward(generated, memory)
            
            #只取最后一个时间步的输出，包含所有可能生成的字的概率
            next_token_logits = output[-1, :, :]  
            
            #使用argmax取出概率最大的字
            next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(0) 
            
            #合并生成的字与之前的文本
            generated = torch.cat((generated, next_token), dim=0)
            if next_token.item() == eos_token_id:  # 假设eos_token_id是结束标记的ID
                break
        return generated

生成式任务中，会多一部分generate用于生成文本序列。

每次循环生成一个时间步的输出。在每个时间步，调用 forward 方法，传递当前生成的序列 generated 作为下一次的输入，并得到下一个时间步的预测结果 output。

然后，从预测结果中选择概率最高的词作为下一个时间步的输入，并将其添加到生成的序列 generated 中。

## 3 Huggingface：Transformer的高级封装与实现

### 3.1 认知Huggingface

Hugging Face是一家领先的人工智能技术落地实践公司，它搭建并开发了围绕自然语言处理（NLP）、大语言模型（LLMs）、多模态模型等各个人工智能领域的一系列落地应用开源框架，其中最为著名的是开源库Transformers。
    
<center><img src="https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/transformer/20.png" alt="描述文字"></center>

随着人工智能领域的发展，模型的体量逐渐增长、数据的体量逐渐增长、在实践中应用复杂的人工智能算法的需求日益增加，越来越多的开发者会倾向于直接使用经过预训练和封装的成熟NLP算法，而非自行构建复杂的transformers或tokernizer架构。Huggingface正是把握住了这一需求的变化，开发了**封装层次极高、调用简单、节约算力、且训练流程清晰明确的Transformers库**，这个库提供了一系列与人工智能相关的预训练模型，如BERT、GPT、T5等，这些模型都是建立在原始Transformer架构基础之上，并对其进行了扩展和优化，以适用于各种各样的NLP任务。与最初由Google提出的Transformer模型相比，Hugging Face的Transformers库提供了更为丰富、易于使用且经过精心优化的模型选择，同时支持跨多种编程语言和平台。

在当代NLP领域的应用中，Huggingface的transformer库有以下三大优势：
    
- **高层次的封装，让Transformer及bert、GPT等更复杂的NLP模型都能被轻松调用**。Huggingface的Transformer宛如NLP领域的sklearn，为低成本、低门槛实践算法铺平了道路。

- **打造了针对特定NLP任务的一系列pipeline供用户使用，让执行复杂任务的流程变得简便**。在NLP的世界中，存在着文本分类、命名实体识别、问答系统、摘要生成、文本生成、机器翻译等不同性质的任务，这些任务涉及到有监督、无监督、半监督等各类标签应用流程，同时每个任务都需要适应于任务本身的文字编码方法、模型架构、预训练权重、以及正确的预处理后处理流程。可以说在NLP的世界中，每个特定NLP任务的执行都是及其复杂而繁琐的。但在Huggingface的transformer库中，pipeline功能可以自动处理任意任务的全流程，用户只需要告诉pipeline当前执行的任务是什么，就可以轻松执行复杂任务。
    
![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/transformer/21.png)
   
- **提供了巨量的预训练模型 & 高级NLP模型的实现**，覆盖了语言理解、生成任务、多模态任务、对话系统以及特定领域的应用。这包括了广泛使用的预训练语言模型如BERT、GPT和RoBERTa，专门用于翻译和文本摘要的序列到序列模型如BART和mT5，以及能处理文本和图像的多模态模型如CLIP和DALL-E。对话模型如DialoGPT和Blenderbot能够支持构建交互式对话系统，而DistilBERT和ALBERT等模型则提供了轻量级的解决方案，适用于资源受限的情况。此外，还有为特定领域或任务定制的模型，如SciBERT和BioBERT等，它们经过预训练能够快速适应相关的NLP任务。你可以在这个页面找到所有能够通过Huggingface的Transformer实现的模型：https://huggingface.co/models

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/transformer/22.png)

今天就让我们来了解下Huggingface的基本框架与运行结构，并使用Huggingface中的Transformer实践两个简单的任务。**注意，huggingface官网访问、模型加载依赖于魔法，建议使用漂亮国全局接口**。

- 安装部署与导入

In [ ]:
#!pip install transformers -i https://pypi.tuna.tsinghua.edu.cn/simple

该代码可以通用于windows、linux和MacOs系统，注意运行pip代码的时候要避开魔法。

In [1]:
import transformers

### 3.2 Huggingface中的功能与模型

在Transformers库当中，库所提供的功能和实际的模型&权重是分割开来的，我们在huggingface的[官方页面](https://huggingface.co)中能够查询到的API是transformers库提供各类功能，而models才是我们可以接入的模型本身。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/transformer/28.png)

在transformers当中，我们通过一系列功能类来调用模型，并决定将模型用于具体的任务。这样的设计主要是基于软件工程的原则，即模块化和解耦。这种设计允许更灵活的研究、开发和部署。功能类完全是由Huggingface官方开发和设计的，而模型和权重则是投放给了开源社区，任意的企业和个人都可以向Huggingface投放自己设计的全新模型和自己训练好的权重。只要遵循Huggingface官方所设置的结构，我们就可以通过transformers库中的功能来调用自己上传的模型。也因此，在huggingface的页面上，我们可以找到30w个不同的模型，这也是Huggingface如此受欢迎的关键原因之一。

在Transformers库当中，要调用一个模型之前，我们至少需要知道**用于调用模型的功能类**以及**实际要调用的模型的名字**。接下来就让我们看看这两部分内容具体如何获得——

- **Huggingface中的功能类板块**

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/transformer/23.png)

> **Models**

Models模块提供了一系列与transformer架构（如BERT、GPT-2、T5等）相关的、用于加载各类模型的功能类，包括但不限于150+基于Transformer的语言模型，100+视觉模型、30+语音模型等等，你可以在[Huggingface官方文档页面](https://huggingface.co/docs/transformers/index)的左侧找到专用于各类模型的类的列表。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/transformer/25.png)

对于语言模型而言，在100多类模型中最值得注意的是——

- **BERT** (Bidirectional Encoder Representations from Transformers): 是一个预训练模型，旨在帮助计算机理解自然语言（即人类使用的语言）。BERT的设计使其能够理解语句中单词的上下文，这对于处理诸如文本分类、命名实体识别、情感分析等任务至关重要。

- **GPT-2** (Generative Pre-trained Transformer 2): 是一个由OpenAI开发的非常强大的文本生成模型。它能生成高质量的文本，并可以用于多种应用，包括写作辅助、对话生成和其他生成任务。

- **RoBERTa** (A Robustly Optimized BERT Pretraining Approach): 是BERT的一个优化版本，通过更仔细的训练策略获得了更好的性能。RoBERTa在许多NLP基准测试中都显示了出色的结果，尤其是在文本分类和情感分析任务上。

- **T5** (Text-to-Text Transfer Transformer): 是一种采用文本到文本框架的模型，它将NLP任务统一为一个文本到文本的格式。T5可以处理多种任务，如翻译、摘要、问答和分类，而且在许多基准测试中表现出色。

- **DistilBERT** (A distilled version of BERT): 是BERT的一个简化版本，旨在减少模型大小，提高速度，同时尽量保持BERT的性能。它适用于资源受限的情况下使用，包括移动设备和小型服务器。

这些模型由于其强大的性能和广泛的适用性，成为了自然语言处理领域的基石。通过预训练和微调，它们可以适用于各种NLP任务，并被用于学术研究和工业应用。

在[每个模型的页面中](https://huggingface.co/docs/transformers/v4.35.0/en/model_doc/bert#bert)，我们可以找到这个模型的系列框架的列表。**注意，这个列表中的类是依据不同需求、从不同角度调用模型的工具，而非模型本身**。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/transformer/26.png)

对Bert模型而言，这些类分别是：

`BertConfig`: 用于存储BERT模型的配置信息，如模型大小、输入维度等。它为模型实例化提供了必要的参数。

`BertTokenizer`: 用于将文本分割成BERT能够理解的token，将词汇转换为模型可以理解的ID。

`BertTokenizerFast`: 是BertTokenizer的更快版本，它使用Rust语言重写，以便更高效地进行分词和编码操作。

`TFBertTokenizer`: 是TensorFlow版本的BERT分词器，为使用TensorFlow框架的用户提供了与BERT模型配合使用的分词功能。

以下是BERT模型结构的变体和专用模型：

`BertModel`: BERT模型的核心，它是预训练模型的主体，可用于特定任务的基础上进行微调。

`BertForPreTraining`: 包含了用于BERT预训练的头部，这个头部同时执行masked language modeling (MLM) 和 next sentence prediction (NSP)。

`BertLMHeadModel`: 这个模型添加了语言模型的头部在BertModel之上，通常用于下游任务中的文本生成。

`BertForMaskedLM`: 专门用于执行masked language modeling的BERT模型。它在训练时遮盖输入的一部分以学习预测这些遮盖的部分。

`BertForNextSentencePrediction`: 这个模型专为预测句子是否为连续句子设计，用于BERT预训练中的下一个句子预测任务。

`BertForSequenceClassification`: 用于序列分类任务，如情感分析，其中模型需要将整个输入序列分类到一个类别。

`BertForMultipleChoice`: 用于多选题任务，可以给出多个可能的答案选项，并预测最可能的一个。

`BertForTokenClassification`: 用于token级别的分类任务，如命名实体识别（NER），其中模型对每个输入token进行分类。

`BertForQuestionAnswering`: 专为问答任务设计，能够预测文本中答案的开始和结束位置。

- **查找模型列表**

现在我们已经知道了bert相关的基本功能类有哪些，那我们就需要下面的页面了：https://huggingface.co/models

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/transformer/28.png)

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/transformer/22.png)

我们可以通过“最多下载”、“最佳趋势”以及搜索等标签来筛选我们需要的模型。只要知道模型的名字，就可以通过transformers中的功能类来调用模型。其中，最常用的模型可以与最常用的功能匹配起来——依然是bert，gpt2，RoBERTa，T5以及DistilBERT。我们可以在对话框中搜索bert，就能够得到一系列关于bert的模型，**点开任意bert相关的模型，即可在说明文档中找到bert相关的全部模型列表**（https://huggingface.co/bert-base-uncased）。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/transformer/29.png)

在Model Card下面可以查看到模型的信息和可用模型的列表名称，在Files and version下面可以下载可用的模型。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/transformer/30.png)

在这些模型中，只有bert-base-chinese与bert-base-multilingual-cased能够支持中文：

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/transformer/31.png)

- **以bert为例，尝试调用bert**

In [78]:
from transformers import BertTokenizer, BertModel #加载能够导入bert模型的工具，最低至少需要BertTokenizer和BertModel

#加载预训练的、专用于bert的分词模型
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese',trust_remote_code=True)

#加载预训练的bert
model = BertModel.from_pretrained("bert-base-chinese",trust_remote_code=True)

text = "虽然今天下雨了，但我拿到了心仪的offer，因此非常开心！"

#将text输入分词和编码模型
encoded_input = tokenizer(text, return_tensors='pt')

#将编码好的文字输入给预训练好的bert
output = model(**encoded_input)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


In [79]:
output #此时bert输出的是针对我们输入的文字进行的判断，注意大部分时候这不是一个生成式的结果

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 1.1622, -0.0790,  1.8173,  ..., -0.6126,  0.6526, -0.5678],
         [-0.0494,  0.9221,  1.3372,  ..., -0.4971, -0.0485,  0.1589],
         [-0.0456, -0.2386,  0.2801,  ...,  0.6094,  0.5508,  0.3712],
         ...,
         [ 1.8941, -0.2364,  0.8497,  ..., -0.2132,  0.6206, -0.0577],
         [ 0.7245, -0.4633,  1.7492,  ..., -0.9154,  0.7376, -0.4374],
         [ 0.7486, -0.3318,  1.1627,  ..., -0.6981,  0.7330, -0.5773]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 0.9999,  0.9999,  0.9996,  0.9229,  0.9680, -0.5962, -0.9971,  0.8632,
          0.9978, -0.9995,  1.0000,  0.9999,  0.9697, -0.9845,  0.9996, -0.9999,
         -0.9962,  0.9950,  0.7646,  0.3401,  0.9999, -1.0000, -0.8772, -0.6877,
         -0.7054,  0.9997,  0.9414, -0.8321, -1.0000,  0.9991,  0.9465,  0.9996,
          0.9964, -1.0000, -1.0000,  0.9912, -0.7883,  0.9957,  0.0435, -0.8159,
         -0.9981, -0.9976,  0.81

通常来说，当我们运行上述代码时，huggingface会开始自动下载对应的模型。**该下载过程需要魔法，建议使用漂亮国去全局接口**，当你是直接下载模型时，trust_remote_code参数要设置为False。但是我们也可以直接将模型通过git，或手动下载到本地，这样就可以通过上述的代码来加载和运行与训练模型了。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/transformer/27.png)

- **尝试调用GPT**

目前为止Huggingface依然只支持调用GPT2，如果要调用GPT3.5甚至4.0则需要直接使用OpenAI所提供的API。

In [80]:
from transformers import GPT2Tokenizer, GPT2Model

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [81]:
output

BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[ 0.1629, -0.2166, -0.1410,  ..., -0.2619, -0.0819,  0.0092],
         [ 0.4628,  0.0248, -0.0785,  ..., -0.0859,  0.5122, -0.3939],
         [-0.0644,  0.1551, -0.6306,  ...,  0.2488,  0.3691,  0.0833],
         ...,
         [-0.5591, -0.4490, -1.4540,  ...,  0.1650, -0.1302, -0.3740],
         [ 0.1400, -0.3875, -0.7916,  ..., -0.1780,  0.1824,  0.2185],
         [ 0.1721, -0.2420, -0.1124,  ..., -0.1068,  0.1205, -0.3213]]],
       grad_fn=<ViewBackward0>), past_key_values=((tensor([[[[-1.0719,  2.4170,  0.9660,  ..., -0.4787, -0.3316,  1.7925],
          [-2.2897,  2.5424,  0.8317,  ..., -0.5299, -2.4828,  1.3537],
          [-2.2856,  2.7125,  2.4725,  ..., -1.4911, -1.8427,  1.6493],
          ...,
          [-3.3203,  2.3325,  2.7061,  ..., -1.1569, -1.5586,  2.4076],
          [-2.9917,  2.2701,  2.1742,  ..., -0.8670, -1.6410,  1.9237],
          [-2.5066,  2.6139,  2.1347,  ..., -0.0627, -2.0542,  1.6568]],

你发现了吗？每次更换模型都需要更换导入的对象和代码，这样就非常麻烦，因此Huggingface又封装了全新的类`AutoClasses`：

> **AutoClasses**

可以通过模型的名字直接定位到使用的模型，是一个通用于大部分模型的功能类。

In [82]:
from transformers import AutoModel, AutoTokenizer

# 对于BERT
bert_model = AutoModel.from_pretrained('bert-base-chinese')
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')

# 对于GPT-2
gpt2_model = AutoModel.from_pretrained('gpt2')
gpt2_tokenizer = AutoTokenizer.from_pretrained('gpt2')

只需要修改模型的名字，就能够实现模型的调用，这比使用Models方便很多。

> **Pipelines**

这是一种高层次的API，用于轻松地完成各种自然语言处理任务，如文本分类、信息提取、问答、摘要、翻译等。pipeline 聚合了模型和预处理，使得用户不需要手动执行文本的编码和解码，模型的加载和输出的解析等步骤。

In [ ]:
from transformers import pipeline

# 创建一个 pipeline，自动加载预训练模型和相应的预处理
#nlp = pipeline("task-name")

# 对文本进行处理，得到结果
#result = nlp("Some input text")

In [84]:
from transformers import pipeline, set_seed

#只需要输入模型的名字，以及要执行的功能，就能够直接实现功能的执行
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("你好呀bert！", max_length=30, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '你好呀bert！的、备決脑八于。这样,'},
 {'generated_text': '你好呀bert！ 東常 情铮真森。真�'},
 {'generated_text': '你好呀bert！Ｑ ﴐ\uf6f0女ﴆ ﴏ�'},
 {'generated_text': '你好呀bert！\uff00Ｆ\uff00３Ｆ Ｆ �'},
 {'generated_text': '你好呀bert！＄\n\nQ: What was his most impressive role or accomplishment?\n\nB:'}]

**在pipelines中，可以选择的任务有**：

"sentiment-analysis": 用于文本情感分类的任务。

"text-generation": 文本生成任务，如自动写作或补全文本。

"ner" (命名实体识别): 识别文本中的实体（如人名、地点、组织）。

"question-answering": 针对给定文本的问题回答任务。

"fill-mask": 填充文本中的掩码（mask）词汇的任务。

"summarization": 自动文本摘要生成任务。

"translation_xx_to_yy": 翻译任务，xx 和 yy 表示不同的语言代码，如 "translation_en_to_fr"。

"text2text-generation": 将文本转换为另一种形式或语言的任务

"zero-shot-classification": 不带训练的直接分类任务，可以为文本分配多个标签。

"conversational": 对话模型，根据对话的历史回应新的对话输入。

"feature-extraction": 提取文本的特征向量。

"text-classification": 文本分类，也被称为主题分类。

"token-classification": 分词层面的分类，如词性标注。

"table-question-answering": 对结构化表格数据进行问题回答的任务。

"translation": 自动翻译任务，通常需要指定源语言和目标语言。

"automatic-speech-recognition": 自动语音识别，将语音转录为文本。

"image-classification": 图像分类任务。

"object-detection": 在图像中识别多个物体及其位置。

"text-to-speech": 文本转语音任务。

**也可以直接调用封装好的pipelines类**

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/transformer/24.png)

In [85]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline

# 指定模型和分词器
model_name = "gpt2"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 直接实例化TextClassificationPipeline类
text_classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

# 对单个句子进行分类
result = text_classifier("I love machine learning!")

# 输出结果
print(result)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[[{'label': 'LABEL_0', 'score': 0.9996962547302246}, {'label': 'LABEL_1', 'score': 0.00030382093973457813}]]


D:\ProgramData\Anaconda3\lib\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
